In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_logs=pd.read_csv(r"../data/logs.csv",sep=";")

In [ ]:
df_logs

In [ ]:
df_logs.head()

In [ ]:
df_logs.tail()

In [ ]:
df_logs.info()

In [ ]:
df_logs.describe()

In [ ]:
df_logs.value_counts("Source")

In [ ]:
df_logs.value_counts("Event group")

In [ ]:
df_logs.value_counts("Event")

In [ ]:
df_logs["Date"]=pd.to_datetime(df_logs["Date"])

#split time and date
def split_date_time(df):
    df["Time"]=df["Date"].dt.time
    df["Date"]=df["Date"].dt.date
    return df

df_logs=split_date_time(df_logs)

# rearrange columns
df_logs=df_logs[["ID","Date","Time","Source","Event group","Event type","Event"]]

In [ ]:
df_logs

In [ ]:
df_logs

In [ ]:
# Count plot of Event group
plt.figure(figsize=(10,6))
sns.countplot(data=df_logs, x="Event group", order=df_logs["Event group"].
value_counts().index)
plt.title("Count of Event group")
plt.xticks(rotation=45)
plt.show()


# Device 1
## for MAC EC:64:C9:91:04:5C

In [ ]:
# for MAC EC:64:C9:91:04:5C
# plot time vs specific event
specific_event="RUT_D8B8_2G: WiFi client connected: EC:64:C9:91:04:5C"
specific_event_2="RUT_D8B8_2G: WiFi client disconnected: EC:64:C9:91:04:5C"

# Filter for both connected and disconnected events for the MAC address
df_event = df_logs[(df_logs["Event"] == specific_event) | (df_logs["Event"] == specific_event_2)]

# Keep only relevant columns and reset index
df_event = df_event[["ID", "Time", "Event"]].reset_index(drop=True)

df_event.info()

In [ ]:
df_event

In [ ]:
# Ensure 'Time_dt' exists before filtering
if 'Time_dt' not in df_event.columns:
    df_event['Time_dt'] = pd.to_datetime(df_event['Time'].astype(str), format='%H:%M:%S')

df_connected = df_event[df_event["Event"].str.contains("connected")]
plt.figure(figsize=(8, 4))
plt.plot(df_connected["Time_dt"], df_connected["ID"], marker='o', color='green')
plt.title("WiFi Client Connected Events")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Convert Time to string for plotting
df_event["Time_str"] = df_event["Time"].astype(str)

plt.figure(figsize=(10,6))
plt.plot(df_event["Time_str"], df_event.index, marker='o')
plt.title(f"Occurrences of Event: {specific_event}")
plt.xlabel("Time")
plt.ylabel("Occurrences")
plt.xticks(rotation=45)
plt.grid()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Ensure Time is in datetime format
df_event['Time_dt'] = pd.to_datetime(df_event['Time'], format='%H:%M:%S')

# Filter events
df_connect = df_event[df_event['Event'].str.contains('connected')].reset_index(drop=True)
df_disconnect = df_event[df_event['Event'].str.contains('disconnected')].reset_index(drop=True)

# Calculate time difference between previous disconnect and current connect
reconnect_diffs = [None]
for i in range(1, len(df_connect)):
    curr_connect_time = df_connect.loc[i, 'Time_dt']
    prev_disconnect = df_disconnect[df_disconnect['Time_dt'] < curr_connect_time]
    if not prev_disconnect.empty:
        disconnect_time = prev_disconnect.iloc[-1]['Time_dt']
        diff = (curr_connect_time - disconnect_time).total_seconds()
    else:
        diff = np.nan
    reconnect_diffs.append(diff)
df_connect['reconnect_diff'] = reconnect_diffs

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(df_connect['Time_dt'], df_connect['ID'], color='green', s=100)
plt.plot(df_connect['Time_dt'], df_connect['ID'], color='lightgreen')

# Annotate lines between connect events with time difference
for i in range(1, len(df_connect)):
    x1, y1 = df_connect.loc[i-1, 'Time_dt'], df_connect.loc[i-1, 'ID']
    x2, y2 = df_connect.loc[i, 'Time_dt'], df_connect.loc[i, 'ID']
    diff = df_connect.loc[i, 'reconnect_diff']
    if diff is not None and not np.isnan(diff):
        mid_x = x1 + (x2 - x1) / 2
        mid_y = y1 + (y2 - y1) / 2
        plt.text(mid_x, mid_y, f"{int(diff)}s", color="black", ha="center", va="bottom", fontweight="bold")

plt.title("WiFi Reconnect Time (Disconnect to Next Connect)")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Ensure 'Time_dt' exists
if 'Time_dt' not in df_event.columns:
    df_event['Time_dt'] = pd.to_datetime(df_event['Time'].astype(str), format='%H:%M:%S')

# Create 'diff_seconds' if missing
if 'diff_seconds' not in df_event.columns:
    df_event['diff_seconds'] = df_event['Time_dt'].diff().dt.total_seconds()
    df_event['diff_seconds'] = df_event['diff_seconds'].fillna(0)

plt.figure(figsize=(12, 5))
ax = plt.gca()

plt.scatter(df_event['Time_dt'], df_event['ID'], s=100, color='blue')
plt.plot(df_event['Time_dt'], df_event['ID'], linestyle='-', color='lightblue')

# Annotate each event with time difference in seconds
for i, row in df_event.iterrows():
    plt.text(row['Time_dt'], row['ID']+0.2, f"{int(row['diff_seconds'])}s", ha='center')

ax.xaxis.set_major_locator(mdates.SecondLocator(interval=50))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

plt.title("WiFi Connection Events Timeline")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# Filter connect and disconnect events
df_connect = df_event[df_event["Event"].str.contains("connected")]
df_disconnect = df_event[df_event["Event"].str.contains("disconnected")]

plt.figure(figsize=(12, 12))
ax = plt.gca()

# Plot connect events
plt.scatter(df_connect['Time_dt'], df_connect['ID'], s=100, color='green', label='Connected')
plt.plot(df_connect['Time_dt'], df_connect['ID'], linestyle='-', color='lightgreen')

# Plot disconnect events
plt.scatter(df_disconnect['Time_dt'], df_disconnect['ID'], s=100, color='red', label='Disconnected')
plt.plot(df_disconnect['Time_dt'], df_disconnect['ID'], linestyle='-', color='salmon')

# Annotate each event with time difference in seconds
for i, row in df_event.iterrows():
    plt.text(row['Time_dt'], row['ID']+0.2, f"{int(row['diff_seconds'])}s", ha='center')

# Format x-axis for time
ax.xaxis.set_major_locator(mdates.SecondLocator(interval=50))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

plt.title("WiFi Connect vs Disconnect Events Timeline")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# Ensure Time_dt is datetime
if 'Time_dt' not in df_event.columns:
    df_event['Time_dt'] = pd.to_datetime(df_event['Time_str'], format='%H:%M:%S')

# Calculate disconnect - previous connect time
disconnect_deltas = []
for idx, row in df_disconnect.iterrows():
    # Find previous connect event
    prev_connect = df_connect[df_connect['Time_dt'] < row['Time_dt']]
    if not prev_connect.empty:
        last_connect_time = prev_connect.iloc[-1]['Time_dt']
        delta = (row['Time_dt'] - last_connect_time).total_seconds()
    else:
        delta = None
    disconnect_deltas.append(delta)

df_disconnect = df_disconnect.copy()
df_disconnect['connect_diff_seconds'] = disconnect_deltas

plt.figure(figsize=(12, 8))
ax = plt.gca()

plt.scatter(df_connect['Time_dt'], df_connect['ID'], s=100, color='green', label='Connected')
plt.plot(df_connect['Time_dt'], df_connect['ID'], linestyle='-', color='lightgreen')

plt.scatter(df_disconnect['Time_dt'], df_disconnect['ID'], s=100, color='red', label='Disconnected')
plt.plot(df_disconnect['Time_dt'], df_disconnect['ID'], linestyle='-', color='salmon')

# Annotate disconnect events with time since previous connect
for i, row in df_disconnect.iterrows():
    if row['connect_diff_seconds'] is not None:
        plt.text(row['Time_dt'], row['ID']+0.2, f"{int(row['connect_diff_seconds'])}s", ha='center', color='black', fontweight='bold')

ax.xaxis.set_major_locator(mdates.SecondLocator(interval=50))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

plt.title("WiFi Disconnect - Connect Time Difference")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.show()

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

# Ensure Time_str and Time_dt columns exist
if 'Time_str' not in df_event.columns:
    df_event['Time_str'] = df_event['Time'].astype(str)
if 'Time_dt' not in df_event.columns:
    df_event['Time_dt'] = pd.to_datetime(df_event['Time_str'], format='%H:%M:%S')

# Filter connect and disconnect events
df_connect = df_event[df_event["Event"].str.contains("connected")].copy()
df_disconnect = df_event[df_event["Event"].str.contains("disconnected")].copy()

# Calculate connect - previous disconnect time
connect_deltas = []
for idx, row in df_connect.iterrows():
    prev_disconnect = df_disconnect[df_disconnect['Time_dt'] < row['Time_dt']]
    if not prev_disconnect.empty:
        last_disconnect_time = prev_disconnect.iloc[-1]['Time_dt']
        delta = (row['Time_dt'] - last_disconnect_time).total_seconds()
    else:
        delta = None
    connect_deltas.append(delta)

df_connect['reconnect_seconds'] = connect_deltas

plt.figure(figsize=(12, 8))
ax = plt.gca()

plt.scatter(df_disconnect['Time_dt'], df_disconnect['ID'], s=100, color='red', label='Disconnected')
plt.plot(df_disconnect['Time_dt'], df_disconnect['ID'], linestyle='-', color='salmon')

plt.scatter(df_connect['Time_dt'], df_connect['ID'], s=100, color='green', label='Connected')
plt.plot(df_connect['Time_dt'], df_connect['ID'], linestyle='-', color='lightgreen')

# Annotate connect events with time since previous disconnect
for i, row in df_connect.iterrows():
    val = row['reconnect_seconds']
    if not pd.isna(val):
        plt.text(row['Time_dt'], row['ID']+0.2, f"{int(val)}s", ha='center', color='black', fontweight='bold')

ax.xaxis.set_major_locator(mdates.SecondLocator(interval=50))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

plt.title("WiFi Reconnect Time After Disconnect")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df_event already exists
# Step 1: Convert Time_str to datetime
df_event['Time_dt'] = pd.to_datetime(df_event['Time_str'], format='%H:%M:%S')

# Step 2: Calculate difference in seconds between consecutive events
df_event['diff_seconds'] = df_event['Time_dt'].diff().dt.total_seconds()
df_event['diff_seconds'].fillna(0, inplace=True)  # first event has no previous

# Step 3: Plot timeline
plt.figure(figsize=(12, 5))

# Plot each event as a marker
plt.scatter(df_event['Time_dt'], df_event['ID'], s=100, color='blue')

# Connect events with lines
plt.plot(df_event['Time_dt'], df_event['ID'], linestyle='-', color='lightblue')

# Optional: annotate with seconds difference
for i, row in df_event.iterrows():
    plt.text(row['Time_dt'], row['ID']+0.2, f"{int(row['diff_seconds'])}s", ha='center')

# Beautify plot
plt.title("WiFi Connection Events Timeline")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)

plt.show()


# Refactoring Repetitive Code

The following cells define functions to avoid repeating the same analysis and plotting code for each device. This makes the notebook cleaner and easier to maintain.


In [ ]:
def plot_connection_timeline(df_logs, mac_address):
    """
    Filters for connection events for a given MAC address and plots their timeline.
    """
    event_string = f"RUT_D8B8_2G: WiFi client connected: {mac_address}"
    df_device_events = df_logs[df_logs["Event"] == event_string].copy()

    if df_device_events.empty:
        print(f"No connection events found for MAC: {mac_address}")
        return

    # Convert time object to datetime for plotting and calculation
    df_device_events['Time_dt'] = pd.to_datetime(df_device_events['Time'].astype(str), format='%H:%M:%S')
    df_device_events = df_device_events.sort_values('Time_dt').reset_index(drop=True)

    # Calculate difference in seconds between consecutive events
    df_device_events['diff_seconds'] = df_device_events['Time_dt'].diff().dt.total_seconds().fillna(0)

    # Plot timeline
    plt.figure(figsize=(12, 5))
    plt.scatter(df_device_events['Time_dt'], df_device_events['ID'], s=100, color='blue')
    plt.plot(df_device_events['Time_dt'], df_device_events['ID'], linestyle='-', color='lightblue')

    # Annotate with seconds difference
    for _, row in df_device_events.iterrows():
        plt.text(row['Time_dt'], row['ID'] + 0.2, f"{int(row['diff_seconds'])}s", ha='center')

    plt.title(f"WiFi Connection Events Timeline for MAC: {mac_address}")
    plt.xlabel("Time")
    plt.ylabel("Event ID")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()


# Device 2
## for MAC 30:C9:22:EF:95:88

In [ ]:
plot_connection_timeline(df_logs, mac_address="30:C9:22:EF:95:88")

# Device 3
## for MAC D8:13:2A:30:5F:88

In [ ]:
plot_connection_timeline(df_logs, mac_address="D8:13:2A:30:5F:88")

# Device 4
## for MAC EC:64:C9:91:08:70

In [ ]:
plot_connection_timeline(df_logs, mac_address="EC:64:C9:91:08:70")

# All Device Graph

In [ ]:
# plot scatter + Line plot of event ID and Time of 4 specific Event
event_1 = "RUT_D8B8_2G: WiFi client connected: EC:64:C9:91:04:5C"
event_2 = "RUT_D8B8_2G: WiFi client connected: 30:C9:22:EF:95:88"
event_3 = "RUT_D8B8_2G: WiFi client connected: D8:13:2A:30:5F:88"
event_4 = "RUT_D8B8_2G: WiFi client connected: EC:64:C9:91:08:70"

df_event_1 = df_logs[df_logs["Event"] == event_1][["ID", "Time"]].reset_index(drop=True)
df_event_2 = df_logs[df_logs["Event"] == event_2][["ID", "Time"]].reset_index(drop=True)
df_event_3 = df_logs[df_logs["Event"] == event_3][["ID", "Time"]].reset_index(drop=True)
df_event_4 = df_logs[df_logs["Event"] == event_4][["ID", "Time"]].reset_index(drop=True)

# Combine all events into a single DataFrame
df_all_events = pd.concat([df_event_1, df_event_2, df_event_3, df_event_4], ignore_index=True)

# Convert Time to string for plotting
df_all_events["Time_str"] = df_all_events["Time"].astype(str)
df_all_events["Time_dt"] = pd.to_datetime(df_all_events["Time_str"], format='%H:%M:%S')
df_all_events = df_all_events.sort_values(by="Time_dt").reset_index(drop=True)
df_all_events['diff_seconds'] = df_all_events['Time_dt'].diff().dt.total_seconds()
df_all_events['diff_seconds'].fillna(0, inplace=True)  # first event
df_all_events.info()

In [ ]:
plt.figure(figsize=(12,     6))
plt.scatter(df_all_events["Time_dt"], df_all_events["ID"], s=100, color='blue')
plt.plot(df_all_events["Time_dt"], df_all_events["ID"], linestyle='-', color='lightblue')
for i, row in df_all_events.iterrows():
    plt.text(row['Time_dt'], row['ID']+0.2, f"{int(row['diff_seconds'])}s", ha='center')
plt.title("WiFi Connection Events Timeline for All Devices")
specific_event_1="RUT_D8B8_2G: WiFi client connected: EC:64:C9:91:04:5C"
specific_event_2="RUT_D8B8_2G: WiFi client connected: 30:C9:22:EF:95:88"
specific_event_3="RUT_D8B8_2G: WiFi client connected: D8:13:2A:30:5F:88"
specific_event_4="RUT_D8B8_2G: WiFi client connected: EC:64:C9:91:08:70"
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Add 'Event' column to each DataFrame
df_event_1 = df_logs[df_logs["Event"] == event_1][["ID", "Time"]].copy()
df_event_1["Event"] = event_1

df_event_2 = df_logs[df_logs["Event"] == event_2][["ID", "Time"]].copy()
df_event_2["Event"] = event_2

df_event_3 = df_logs[df_logs["Event"] == event_3][["ID", "Time"]].copy()
df_event_3["Event"] = event_3

df_event_4 = df_logs[df_logs["Event"] == event_4][["ID", "Time"]].copy()
df_event_4["Event"] = event_4

# Combine all events into a single DataFrame
df_all_events = pd.concat([df_event_1, df_event_2, df_event_3, df_event_4], ignore_index=True)

# Continue with your plotting code
# Convert Time to string and then to datetime for all events
df_all_events["Time_str"] = df_all_events["Time"].astype(str)
df_all_events["Time_dt"] = pd.to_datetime(df_all_events["Time_str"], format='%H:%M:%S')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# If 'Time' is a datetime.time object, convert to string first
df_connected['Time_str'] = df_connected['Time'].astype(str)
# Convert to datetime using a dummy date
df_connected['Time_dt'] = pd.to_datetime(df_connected['Time_str'], format='%H:%M:%S')

plt.figure(figsize=(8, 4))
plt.plot(df_connected['Time_dt'], df_connected['ID'], marker='o', color='green')
plt.title("WiFi Client Connected Events")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Export processed logs and event DataFrames
import os
os.makedirs('../src/static',exist_ok=True)
df_logs.to_csv('log_dashboard/data/processed_logs.csv', index=False)
df_event.to_csv('log_dashboard/data/processed_event.csv', index=False)

# Save connect/disconnect timeline plots
# Convert 'Time' to string, then to datetime for plotting

df_connected['Time_str'] = df_connected['Time'].astype(str)
df_connected['Time_dt'] = pd.to_datetime(df_connected['Time_str'], format='%H:%M:%S')
plt.figure(figsize=(8, 4))
plt.plot(df_connected['Time_dt'], df_connected['ID'], marker='o', color='green')
plt.title("WiFi Client Connected Events")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig('log_dashboard/app/static/connect_timeline.png')
plt.close()

# Make sure df_event or df_logs is defined and contains the relevant data
df_disconnected = df_logs[df_logs['Event'].str.contains('disconnected')].copy()

# Now you can safely process and plot
df_disconnected['Time_str'] = df_disconnected['Time'].astype(str)
df_disconnected['Time_dt'] = pd.to_datetime(df_disconnected['Time_str'], format='%H:%M:%S')
plt.figure(figsize=(8, 4))
plt.plot(df_disconnected['Time_dt'], df_disconnected['ID'], marker='o', color='red')
plt.title("WiFi Client Disconnected Events")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig('log_dashboard/app/static/disconnect_timeline.png')
plt.close()

df_disconnected['Time_str'] = df_disconnected['Time'].astype(str)
df_disconnected['Time_dt'] = pd.to_datetime(df_disconnected['Time_str'], format='%H:%M:%S')
plt.figure(figsize=(8, 4))
plt.plot(df_disconnected['Time_dt'], df_disconnected['ID'], marker='o', color='red')
plt.title("WiFi Client Disconnected Events")
plt.xlabel("Time")
plt.ylabel("Event ID")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig('log_dashboard/app/static/disconnect_timeline.png')
plt.close()

plt.figure(figsize=(10,6))
sns.countplot(data=df_logs, x="Event group", order=df_logs["Event group"].value_counts().index)
plt.title("Count of Event group")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('log_dashboard/app/static/event_group_count.png')
plt.close()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# List of specific disconnected events
specific_disconnected_events = [
    "RUT_D8B8_2G: WiFi client disconnected: EC:64:C9:91:04:5C",
    "RUT_D8B8_2G: WiFi client disconnected: 30:C9:22:EF:95:88",
    "RUT_D8B8_2G: WiFi client disconnected: D8:13:2A:30:5F:88",
    "RUT_D8B8_2G: WiFi client disconnected: EC:64:C9:91:08:70"
]

# Filter for only those specific disconnected events
df_disconnected = df_logs[df_logs['Event'].isin(specific_disconnected_events)].copy()

if not df_disconnected.empty:
    df_disconnected['Time_str'] = df_disconnected['Time'].astype(str)
    df_disconnected['Time_dt'] = pd.to_datetime(df_disconnected['Time_str'], format='%H:%M:%S')
    plt.figure(figsize=(8, 4))
    plt.plot(df_disconnected['Time_dt'], df_disconnected['ID'], marker='o', color='red')
    plt.title("Specific WiFi Client Disconnected Events")
    plt.xlabel("Time")
    plt.ylabel("Event ID")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('log_dashboard/app/static/disconnect_timeline.png')
    plt.close()
else:
    plt.figure(figsize=(8, 4))
    plt.text(0.5, 0.5, 'No specific disconnect events found', ha='center', va='center', fontsize=16)
    plt.axis('off')
    plt.savefig('log_dashboard/app/static/disconnect_timeline.png')
    plt.close()

In [ ]:
df_disconnected